This Jupyter Notebook was tested with the following Python packages:
- torch==2.5.1
- transformers==4.48.1
- datasets==3.2.0

# BERT-style Encoder Language Models

## Overview

BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based model designed to pre-train deep bidirectional representations by jointly conditioning on both left and right context in all layers. This allows the model to understand the context of a word based on its surroundings, making it highly effective for various NLP tasks. Thus it can be seen as a model producing a contextualized embedding that can be used as a replacement for e.g., GloVe.

## Key Features

1. **Bidirectional Context**: Unlike traditional models that read text sequentially (left-to-right or right-to-left), BERT reads the entire sequence of words at once, allowing it to understand the context of a word based on both its preceding and following words.

2. **Pre-training Objectives**:
    - **Masked Language Model (MLM)**: Randomly masks some of the tokens in the input, and the objective is to predict the original vocabulary id of the masked word based only on its context.
    - **Next Sentence Prediction (NSP)**: Predicts whether a given pair of sentences is consecutive in the original text, helping the model understand sentence relationships.

## Applications

BERT can be fine-tuned for various downstream tasks, including:

- **Text Classification**: Sentiment analysis, spam detection, etc.
- **Question Answering**: Extracting answers from a given context.
- **Named Entity Recognition (NER)**: Identifying entities like names, dates, and locations in text.
- **Text Summarization**: Generating concise summaries of longer texts.


We first install the needed packages:

In [1]:
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    DataCollatorWithPadding,
    RobertaForSequenceClassification,
    RobertaModel,
    RobertaTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
    EvalPrediction,
)

We will again use the SST2 dataset. Let's load it:

In [3]:
# Load the dataset
dataset = load_dataset("stanfordnlp/sst2")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})


When dealing with pre-trained models, you will have to download the weights from somewhere.\
These weights have to fit the model architecture, therefore we rely on a library for the models and its weights.\
We here use the Hugging Face transformers library for loading the model and its tokenizer.

There are so many encoder language models; here we use RoBERTa which is an optimized (in terms of hyperparameters) version of BERT.

In [5]:
# Load the tokenizer and model
'''"roberta-base"是一个由Facebook AI研究院开发的预训练语言模型，它是BERT模型的改进版本。
通过使用这些预训练的模型和分词器，您可以快速开始处理各种NLP任务，如文本分类、命名实体识别、问答系统等'''
'''这段代码为后续的NLP任务做好了准备，您可以使用tokenizer来处理输入文本，
然后将处理后的输入传递给roberta模型进行进一步的处理或微调。'''
tokenizer: RobertaTokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained("roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We can inspect the model's architecture by printing it:

In [6]:

print(roberta)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

As before, we prepare the dataset, but this time the tokenizer does most of the work converting text to token ids for the loaded model:

In [7]:
# Activate if you want to use a subset of the dataset for faster training
USE_SUBSET = False

In [8]:
# Tokenize the dataset
def tokenize(examples, tokenizer):
    # tokenizer的主要功能是将文本转换为模型可以理解的数字序列
    # 这是输入数据。examples是一个字典或类似字典的对象，包含了多个样本。
    # ["sentence"]表示我们正在访问每个样本的"sentence"字段，这里面存储着需要被处理的文本。
    # truncation=True启用了截断功能。如果输入的句子长度超过了模型的最大输入长度（对于RoBERTa通常是512个token），tokenizer会自动截断句子以适应模型。
    # 注意力掩码（attention mask）通常用于指示哪些token是真实的输入，哪些是填充（padding）。
    # 将此参数设置为False意味着tokenizer不会返回注意力掩码。
    return tokenizer(examples["sentence"], truncation=True, return_attention_mask=False)


# Prepare the datasets
# dataset.map()：map 是数据集对象的一个方法，用于对数据集中的每个样本应用一个函数。这个方法会返回一个新的数据集，其中包含了经过处理的样本。
# tokenize：这是我们之前定义的函数，用于对输入的文本进行分词处理。它被作为第一个参数传递给 map 方法，表示要对每个样本应用这个函数。
# fn_kwargs 参数用于指定要传递给映射函数（在这个例子中是 tokenize 函数）的额外关键字参数
# dict(tokenizer=tokenizer)创建了一个字典，其中包含一个键值对。键是 'tokenizer'，值是之前定义的 tokenizer 对象。
'''当 dataset.map() 调用 tokenize 函数时，它会自动将 examples 作为第一个参数传递。
然后，它会使用 fn_kwargs 中指定的参数作为额外的关键字参数。
因此，tokenize 函数会收到两个参数：examples（来自数据集）和 tokenizer（来自 fn_kwargs）。'''
# 设置 batched=True 意味着 map 方法会以批处理模式运行。
# num_proc=10设置为 10 意味着将使用 10 个并行进程来处理数据集。
tokenized_datasets = dataset.map(tokenize, fn_kwargs=dict(tokenizer=tokenizer), batched=True, num_proc=10)

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# Use a subset for quick training
# 如果USE_SUBSET为True，则会使用数据集的一个小子集。这通常用于快速测试或调试。
if USE_SUBSET:
    # shuffle(seed=42): 使用固定的随机种子42对数据集进行随机打乱。select(range(1000)): 从打乱后的数据集中选择前1000个样本
    train_dataset = train_dataset.shuffle(seed=42).select(range(1000))
    val_dataset = val_dataset.shuffle(seed=42).select(range(100))
    test_dataset = test_dataset.shuffle(seed=42).select(range(100))
# 这两行代码用于打印训练集的特征和第一个样本，以便检查数据处理的结果。
# Print the features
print(train_dataset.features)
# Print the first example
print(train_dataset[0])

Map (num_proc=10):   0%|          | 0/67349 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/872 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/1821 [00:00<?, ? examples/s]

{'idx': Value(dtype='int32', id=None), 'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'positive'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
{'idx': 0, 'sentence': 'hide new secretions from the parental units ', 'label': 0, 'input_ids': [0, 37265, 92, 3556, 2485, 31, 5, 20536, 2833, 1437, 2]}


## Task
Feed the first training example through the LM.
>Note: The model takes the token ids with the argument `input_ids`.

In [9]:
# Call model on the first example
# train_dataset[0]: 获取训练集的第一个样本。["input_ids"]: 提取该样本的输入ID序列。torch.tensor(): 将输入ID转换为PyTorch张量。
#.unsqueeze(0): 在第0维添加一个维度，将形状从 [sequence_length] 变为 [1, sequence_length]，这是因为模型期望的输入是一个批次。
# output = roberta(input_ids=...)将处理后的输入传递给RoBERTa模型。模型返回一个包含多个输出的对象。
output = roberta(input_ids=torch.tensor(train_dataset[0]["input_ids"]).unsqueeze(0))
'''last_hidden_state: 这是模型最后一层的隐藏状态。
打印其形状，通常为 [batch_size, sequence_length, hidden_size]。
batch_size 在这里是1。
sequence_length 是输入序列的长度。
hidden_size 是模型的隐藏层大小（对于RoBERTa-base通常是768）。'''
print(output.last_hidden_state.shape)
'''pooler_output: 这是序列的池化表示，通常用于分类任务。
打印其形状，通常为 [batch_size, hidden_size]。
batch_size 在这里是1。
hidden_size 与上面相同。'''
print(output.pooler_output.shape)

torch.Size([1, 11, 768])
torch.Size([1, 768])


The model's output is a contextualized representation of the input, and therefor can be used as such in your neural network:

In [10]:
class SentimentAnalysisModel(torch.nn.Module):
    # base_model：预训练的transformer模型（如RoBERTa）
    # freeze_embedder：布尔值，决定是否冻结基础模型的参数。
    def __init__(self, base_model, freeze_embedder=False):
        super().__init__()
        # We are using the transformer model as base_model
        self.base_model = base_model
        # Freeze the base_model
        if freeze_embedder:
            # self.base_model.parameters() 返回基础模型（如RoBERTa）的所有可学习参数。这个循环遍历每一个参数。
            for param in self.base_model.parameters():
                # 当一个参数的 requires_grad 为 False 时，PyTorch 不会为该参数计算梯度，也不会在反向传播过程中更新这个参数
                param.requires_grad = False
        # We add a linear layer on top of the base_model
        # 添加一个线性层作为分类器。输入维度是基础模型的隐藏层大小，输出维度是2（对应两个情感类别）
        '''预训练的基础模型（如RoBERTa）是为通用语言理解任务设计的，而不是专门为情感分析设计。
分类器层将通用的语言表示转化为特定任务（情感分析）的输出。
基础模型的输出通常是高维的（例如，768维）。
情感分析任务需要的是低维输出（在这个例子中是2维，对应两种情感）。
分类器实现了从高维到低维的映射。'''
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, 2)
        # Initialize the classifier weights
        # 使用Xavier均匀分布初始化分类器的权重。
        # 将分类器的偏置初始化为零。
        torch.nn.init.xavier_uniform_(self.classifier.weight) # Xavier uniform initialization
        torch.nn.init.zeros_(self.classifier.bias) # Initialize bias with zeros

    def forward(self, **model_inputs):# **用于接收任意数量的关键字参数和解包字典
        '''**model_inputs允许传入任意关键字参数，model_inputs 通常是一个字典，包含了模型所需的各种输入。
首先通过基础模型处理输入。
然后使用基础模型的pooler_output作为句子表示，传入分类器。
返回分类器的输出。'''
        # model_inputs is a dict
        # Pass the inputs to the model to produce an embedding
        base_model_output = self.base_model(**model_inputs)
        # Pass the embedding through the classifier
        # here we use the pooler_output as the representation of the sentence (depends on the model)
        output = self.classifier(base_model_output.pooler_output)
        return output
'''这个模型的工作流程是：
输入文本通过预训练的transformer模型（如RoBERTa）进行处理。
使用transformer模型的池化输出作为文本的表示。
将这个表示传入一个简单的线性分类器。
分类器输出两个数值，对应两种情感类别的得分。
这种设计允许利用预训练模型的强大特征提取能力，同时通过微调或添加任务特定层来适应情感分析任务。
freeze_embedder选项提供了灵活性，可以选择是否更新预训练模型的参数，这在不同的应用场景中可能会有不同的效果。'''


'这个模型的工作流程是：\n输入文本通过预训练的transformer模型（如RoBERTa）进行处理。\n使用transformer模型的池化输出作为文本的表示。\n将这个表示传入一个简单的线性分类器。\n分类器输出两个数值，对应两种情感类别的得分。\n这种设计允许利用预训练模型的强大特征提取能力，同时通过微调或添加任务特定层来适应情感分析任务。\nfreeze_embedder选项提供了灵活性，可以选择是否更新预训练模型的参数，这在不同的应用场景中可能会有不同的效果。'

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# 创建 SentimentAnalysisModel 的实例。
# roberta 是预先加载的RoBERTa模型，作为基础模型传入。
# .to(device) 将模型移动到之前定义的设备（GPU或CPU）上。
model = SentimentAnalysisModel(roberta, freeze_embedder=False).to(device) # Set to True to freeze the embedder for faster training
# 这行代码对模型进行了一个简单的测试。
# train_dataset[0]["input_ids"] 获取训练集中第一个样本的输入ID。
# torch.tensor([...]) 将输入ID转换为PyTorch张量。
# .device=device 确保输入数据在正确的设备上（与模型相同）。
# 整个表达式将一个样本输入模型，测试模型的前向传播。
model(input_ids=torch.tensor([train_dataset[0]["input_ids"]], device=device))

'''这段代码的目的和作用：
设备优化：通过选择合适的设备（GPU或CPU），确保模型能够高效运行。
模型初始化：创建并配置情感分析模型，为后续的训练做准备。
灵活性：通过 freeze_embedder 参数，提供了是否冻结基础模型的选项，允许在不同场景下进行实验。
初步测试：通过运行一个样本，验证模型的基本功能是否正常，包括数据处理和前向传播。
调试准备：这种设置有助于快速发现任何初始化或兼容性问题。'''

'这段代码的目的和作用：\n设备优化：通过选择合适的设备（GPU或CPU），确保模型能够高效运行。\n模型初始化：创建并配置情感分析模型，为后续的训练做准备。\n灵活性：通过 freeze_embedder 参数，提供了是否冻结基础模型的选项，允许在不同场景下进行实验。\n初步测试：通过运行一个样本，验证模型的基本功能是否正常，包括数据处理和前向传播。\n调试准备：这种设置有助于快速发现任何初始化或兼容性问题。'

So far we have only passed the token ids to the model.\
However, there are more inputs for transformer-based models which you need to be aware of:

1. **input_ids**: Token IDs to be fed to the model.
2. **attention_mask**: Mask to avoid performing attention on padding token indices.
3. **token_type_ids**: Segment token indices to indicate different portions of the inputs (used in models like BERT for tasks like question answering).
4. **position_ids**: Indices of positions of each input sequence token in the position embeddings.

If you don't pass them into the model, the library will take care of it!\
However, if you perform padding during batching, the model might not know where padding was applied and therefore you will also have to provide the `attention_mask`!

Having the processed data and the model in place, we create the appropriate dataloader:

In [13]:
# features 是一个列表，包含了多个样本（每个样本是一个字典）
def collate_fn(features):
    # We need to pad the input to make sure all sentences have the same length
    # 将每个样本的 input_ids 转换为张量。
    # 使用 pad_sequence 将不同长度的序列填充到相同长度。
    # batch_first=True 确保输出的形状是 (batch_size, sequence_length)。
    # padding_value 使用tokenizer的pad_token_id。pad_token_id在某些模型中，它可能是0或1，而在其他模型中可能是不同的值。
    input_ids = torch.nn.utils.rnn.pad_sequence(
        [torch.tensor(f["input_ids"]) for f in features],
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
    )
    if "attention_mask" in features[0]:
        # If the features contain attention_mask, we should pad them as well
        attention_mask = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(f["attention_mask"]) for f in features],
            batch_first=True,
            padding_value=0,
        )
    else:
        '''input_ids != tokenizer.pad_token_id:
这个操作会比较 input_ids 张量中的每个元素与 tokenizer.pad_token_id 的值。
结果是一个布尔张量,.int() 转换:将布尔张量转换为整数张量。True 变为 1，False 变为 0。
得到的 attention_mask 是一个与 input_ids 形状相同的张量。
值为 1 的位置表示实际的 token（模型应该关注的部分）。
值为 0 的位置表示填充 token（模型应该忽略的部分）。
指导模型注意力:
告诉模型哪些位置包含实际的输入信息，哪些位置是填充。
模型在计算自注意力时会使用这个掩码，确保不会关注到填充的部分。'''
        # We need to create an attention mask from input_ids
        attention_mask = (input_ids != tokenizer.pad_token_id).int()
    labels = torch.tensor([f["label"] for f in features])
    # 将处理后的数据组织成一个字典。
    batch = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }
    return batch

# DataLoader 是 PyTorch 中用于加载数据的工具
#batch_size: 训练集为2，验证集为8。这决定了每次迭代处理的样本数。
train_dataloader = DataLoader(
    train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn
)
val_dataloader = DataLoader(
    val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

for batch in train_dataloader:
    print(batch)
    print(batch.keys())
    break

{'input_ids': tensor([[    0,   627,  5581,     9,   209,    80, 17205,  2415,  2156,     8,
           190,     5,   498,    11,    61,    51,  3033,  1437,     2],
        [    0, 23702,  1461,  1437,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=torch.int32), 'labels': tensor([1, 0])}
dict_keys(['input_ids', 'attention_mask', 'labels'])


Instead of implementing the collate function ourselbes, the transformers library provides them for different applications as well:

In [14]:
# with_format 方法用于指定数据集应该返回的列
# DataCollatorWithPadding(tokenizer, padding="longest")是 Hugging Face Transformers 库提供的一个工具，专门用于处理文本数据的批处理
# DataCollatorWithPadding 是针对 transformer 模型优化的，可能比自定义函数更高效;而且不用自己写collate_fn
# 它主要负责将不同长度的序列填充到相同长度，以便能够组成一个批次
#　tokenizer是您使用的 tokenizer 对象，通常是与您的模型相对应的 tokenizer
# padding="longest":指定了填充策略。"longest" 表示在每个批次中，所有序列都会被填充到该批次中最长序列的长度
train_dataloader = DataLoader(
    train_dataset.with_format(columns=["input_ids", "label"]),
    batch_size=32,
    shuffle=True,
    collate_fn=DataCollatorWithPadding(tokenizer, padding="longest"),
)

for batch in train_dataloader:
    print(batch)
    print(batch.keys())
    break

{'input_ids': tensor([[    0,   560,  7798,  ...,     1,     1,     1],
        [    0,  1090,   991,  ...,     1,     1,     1],
        [    0,  1116,   130,  ...,     1,     1,     1],
        ...,
        [    0,    18,    10,  ...,     1,     1,     1],
        [    0, 12473,    65,  ...,     1,     1,     1],
        [    0,   354,  8045,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 1])}
dict_keys(['input_ids', 'attention_mask', 'labels'])


In [15]:
val_dataloader = DataLoader(
    val_dataset.with_format(columns=["input_ids", "label"]),
    batch_size=128,
    shuffle=False,
    collate_fn=DataCollatorWithPadding(tokenizer, padding="longest"),
)

for batch in val_dataloader:
    print(batch)
    print(batch.keys())
    break

{'input_ids': tensor([[    0,   405,   128,  ...,     1,     1,     1],
        [    0,   879,  4825,  ...,     1,     1,     1],
        [    0, 37984,   201,  ...,     1,     1,     1],
        ...,
        [    0,   102,  1569,  ...,     1,     1,     1],
        [    0, 17075, 21871,  ...,     1,     1,     1],
        [    0,   179,     5,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 

We also write an evaluation function that computes the accuracy of all samples in a dataloader using the model:

In [16]:
# Evaluate the model
def evaluate(model, dataloader, device=None):
    # 在评估过程中不需要计算梯度，这可以节省内存并加速计算
    with torch.no_grad():
        # model.eval() 是 PyTorch 中一个非常重要的方法调用，用于将模型设置为评估模式
        model.eval()
        total = 0
        correct = 0
        # 使用 tqdm 显示进度条
        # dataloader: 这是要迭代的对象，在这里是 PyTorch 的 DataLoader 实例。
        # desc="Evaluation": 设置进度条的描述文本，这里显示为 "Evaluation"。
        # leave=False: 进度条完成后是否保留。设置为 False 意味着进度条会在完成后消失
        for batch in tqdm(dataloader, desc="Evaluation", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            # 使用模型对输入数据进行前向传播。
            # input_ids: 输入序列的token ID。
            # attention_mask: 指示哪些位置是有效输入（非填充）。
            # outputs: 模型的输出，通常是每个类别的预测分数（logits）
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # torch.argmax: 返回指定维度上最大值的索引。
            # dim=1: 在第二个维度（类别维度）上取最大值。
            # predicted: 包含每个样本预测类别的张量。
            #　例如predicted = tensor([1, 1, 0, 1])，predicted 张量包含了每个样本的预测类别：1 表示模型预测该评论为正面；0 表示模型预测该评论为负面
            predicted = torch.argmax(outputs, dim=1)
            # tensor.size(0) 特指返回张量第一个维度（通常是批次大小）的大小。
            # 在大多数 PyTorch 数据加载器中，第一个维度通常表示批次大小。对于分类任务，labels 通常是一个一维张量，其长度等于批次大小。
            total += labels.size(0)
            #　.item() 方法:将单元素张量转换为 Python 标量。这是必要的，因为我们要将结果加到 Python 的整数变量 correct 上
            correct += (predicted == labels).sum().item()
    return correct / total

In [17]:
print(f"Evaluation accuracy on validation data: {evaluate(model, val_dataloader, device=device)}")

Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluation accuracy on validation data: 0.4690366972477064


Next, we need a training loop that is just the same as before.\
We have to make sure to feed in the correct arguments to our model:

In [18]:
# 使用 AdamW 优化器，这是 Adam 优化器的一个变体，具有权重衰减修正
# model.parameters(): 指定要优化的参数，这里是模型的所有参数
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.01)
loss_fn = torch.nn.CrossEntropyLoss()
#　这意味着整个训练数据集将被遍历 3 次。
num_epochs = 3
# 创建一个字典来存储和跟踪训练过程中的指标
metric_dict = {"Loss": "-", "Val Acc": evaluate(model, val_dataloader, device=device)}
# total: 设置为总批次数（轮数 * 每轮批次数）；DataLoader 被设计为一个可迭代对象，每次迭代返回一个批次的数据，所以len(train_dataloader)是训练数据加载器中的批次数量
# unit: 进度单位设为 "batch"
with tqdm(
    total=num_epochs * len(train_dataloader), desc="Training", unit="batch"
) as pbar:
    for epoch in range(num_epochs):
        # Set the model in training mode
        model.train()
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(output, labels)
            # 清除之前的梯度
            optimizer.zero_grad()
            # 反向传播，计算梯度
            loss.backward()
            # 更新模型参数
            optimizer.step()
            #　更新损失值
            # loss.item() 用于将单个元素的 PyTorch 张量转换为 Python 标量
            metric_dict["Loss"] = loss.item()
            # 更新进度条显示的指标
            pbar.set_postfix(metric_dict)
            # 推进进度条
            pbar.update(1)
        metric_dict["Val Acc"] = evaluate(model, val_dataloader, device=device)
        pbar.set_postfix(metric_dict)

Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/6315 [00:00<?, ?batch/s]

Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

Alternatively, the transformers library makes it simple to use LMs as it includes task-specific models for finetuning:

In [19]:
# RobertaForSequenceClassification model can be used for text classification tasks like sentiment analysis
# It has a sequence classification head, that is a linear layer on top of the RoBERTa model that outputs a classification label
# RobertaForSequenceClassification是 Transformers 库中的一个类，专门用于序列分类任务的 RoBERTa 模型
# 它在 RoBERTa 基础模型之上添加了一个分类头（通常是一个线性层）
# .from_pretrained():是一个类方法，用于加载预训练的模型权重。它不仅加载权重，还配置模型架构以匹配预训练模型。
#　roberta-base是要加载的预训练模型的标识符，用于识别和加载特定的预训练模型
# num_labels=2 指定分类任务的类别数量。在这个例子中，设置为 2 表示这是一个二分类任务。
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

'''这个例子展示了如何加载模型和相应的分词器，并使用它们进行简单的预测。
总之，这行代码是使用预训练语言模型进行下游任务的典型起点，
它利用了迁移学习的强大功能，为特定的分类任务提供了一个强大的基础。'''

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'这个例子展示了如何加载模型和相应的分词器，并使用它们进行简单的预测。\n总之，这行代码是使用预训练语言模型进行下游任务的典型起点，\n它利用了迁移学习的强大功能，为特定的分类任务提供了一个强大的基础。'

You can also use their Trainer API so that you don't have to implement the training loop again and again:

In [20]:
# Define a function to compute the metrics
# 这是一个名为 compute_metrics 的函数，用于计算模型预测的准确率
def compute_metrics(pred_and_label: EvalPrediction):
    return {
        # predictions.argmax(axis=-1)获取预测结果中概率最高的类别索引;argmax 在最后一个轴上操作，通常对应于类别维度
        # == pred_and_label.label_ids将预测的类别与真实标签进行比较;生成一个布尔数组，True 表示预测正确，False 表示预测错误
        # .mean()计算布尔数组的平均值，即正确预测的比例;.item()将结果从 PyTorch 张量转换为 Python 标量
        "accuracy": (pred_and_label.predictions.argmax(axis=-1) == pred_and_label.label_ids)
        .mean()
        .item()
    }
# 返回值： 函数返回一个字典 {"accuracy": 计算得到的准确率}

# Define the training arguments
# TrainingArguments 对象用于配置模型训练的各种参数;是 Hugging Face Transformers 库中的一个类，用于设置训练过程中的各种选项

training_args = TrainingArguments(
    # 指定训练输出（如检查点、日志等）的保存目录
    output_dir="./results",
    # 设置评估策略为每个 epoch 结束后进行评估。
    evaluation_strategy="epoch",
    # 设置日志记录策略为每个 epoch 结束后记录。
    logging_strategy="epoch",
    # 使用 Hugging Face 实现的 AdamW 优化器。
    optim="adamw_hf",
    # 设置学习率为 5e-5，这是一个常用的微调学习率。
    learning_rate=5e-5,
    # 每个设备（GPU/CPU）的训练批次大小为 32。
    per_device_train_batch_size=32,
    # 每个设备的评估批次大小为 128。评估时通常可以使用更大的批次。
    per_device_eval_batch_size=128,
    # 设置训练轮数为 3 个 epoch
    num_train_epochs=3,
    # 设置权重衰减为 0.01，用于正则化。
    weight_decay=0.01,
    # 不使用 CPU 进行训练，默认使用 GPU（如果可用）
    use_cpu=False,
    # 在训练开始前进行一次评估。
    eval_on_start=True,
    # 不保存模型检查点，以节省磁盘空间。
    save_strategy="no",  # We will not save the model for now to save disk space
)

# Define the trainer
# 这段代码使用 Hugging Face 的 Trainer 类来设置、训练和评估模型。
trainer = Trainer(
    # 之前定义的模型（可能是 RoBERTa 或其他 Transformer 模型）。
    model=model,
    # args: 之前定义的 TrainingArguments 对象。
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # processing_class: 设置为 tokenizer，用于处理输入数据（如填充批次）
    processing_class=tokenizer,  # enables padding of batches
    # 之前定义的计算指标的函数。
    compute_metrics=compute_metrics,
)

# Train the model
# 这行代码启动模型训练过程。Trainer 会根据之前设置的参数进行训练
trainer.train()

# Evaluate the model
# 这行代码在训练结束后对模型进行最终评估。
trainer.evaluate()

e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
e:\Uni Stuttgart\课程\Intro to DL\my_env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.694606,0.490826


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 2.98 GiB is allocated by PyTorch, and 376.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Text Generation Language Models with Prompting

Text generation language models, such as GPT-3, can be used with prompting to perform various natural language processing tasks, including sentiment analysis.\
Prompting involves providing the model with a specific input or "prompt" that guides it to generate the desired output.\
This technique leverages the model's pre-trained knowledge to perform tasks without requiring additional fine-tuning.

#### Applying Prompting to Sentiment Analysis

To use a text generation model for sentiment analysis, you can craft a prompt that instructs the model to classify the sentiment of a given text. The prompt should be designed to elicit a response that indicates whether the sentiment is positive or negative.


The transformers library makes it simple to load a text generation model.
>Note that they can be quite large and potentially do not fit or run slowly on your CPU/RAM.

In [ ]:
# Load the pipeline
# 这是 Hugging Face Transformers 库中的一个高级 API，用于快速设置和使用预训练模型进行各种 NLP 任务
pipe = pipeline(
    # 指定任务类型为文本生成,这意味着模型将被用于生成文本，而不是分类或其他任务
    "text-generation",
    # 指定使用的模型;这是微软开发的 Phi-3.5-mini-instruct 模型，是一个相对较小但功能强大的指令调优语言模型
    model="microsoft/Phi-3.5-mini-instruct",
    trust_remote_code=True,  # Trust the remote code; this is required for some models, but always check the code first!
    device=device,  # Set this to "cuda" for GPU acceleration if available
    # 设置模型使用的数据类型为 bfloat16
    torch_dtype=torch.bfloat16,  # Use bfloat16 for less memory usage and faster inference
)

### Input Format for Chat-Based Decoder Models

Chat-based decoder models, such as GPT-3, typically require inputs in a specific format to generate coherent and contextually relevant responses.\
The input format generally consists of a sequence of messages, each with a role and content.\
The roles depend on the model, and often are "system", "user", or "assistant".

#### Example Input Format

```python
messages = [
    {
        "role": "system",
        "content": "Your task is to perform sentiment analysis. Classify the sentiment of the provided text into 'negative' or 'positive' and return only this label.",
    },
    {"role": "user", "content": "This movie was the worst movie I have ever seen."},
]
```

#### Key Components

1. **Role**: Indicates the role of the message sender. Common roles include:
   - `system`: Provides instructions or context for the conversation.
   - `user`: Represents the input from the user.
   - `assistant`: Represents the response from the model.

2. **Content**: The actual text of the message.

In [ ]:
messages = [
    {
        "role": "system",
        "content": "Your task is to perform sentiment analysis. Classify the sentiment of the provided text into 'negative' or 'positive' and return only this label.",
    },
    {"role": "user", "content": "This movie was the worst movie I have ever seen."},
]

generation_args = {
    #  限制生成的最大 token 数为 3，因为我们只需要一个简短的标签。
    "max_new_tokens": 3,  # maximum number of tokens to generate
    # 设为 False，只返回新生成的文本，而不是完整的回复。
    "return_full_text": False,
    # temperature=0.0 和 do_sample=False：最确定性的设置。模型总是选择最可能的下一个词。适合需要一致和精确答案的任务，如分类或特定信息提取。
    # temperature>0.0 和 do_sample=True：引入随机性，使输出更加多样化。温度越高，输出越随机。适合创意写作或需要多样化回答的场景。
    "temperature": 0.0,  # temperature for sampling (on if do_sample=True)
    "do_sample": False,  # whether to sample from the output distribution
}
# 将消息和生成参数传递给管道。使用 **generation_args 展开参数字典。
output = pipe(messages, **generation_args)
print(output)
'''总的来说，这段代码展示了如何使用通用语言模型执行特定的 NLP 任务（在这里是情感分析），
通过精心设计的提示和参数设置来引导模型产生所需的输出。这种方法的灵活性使得同一个模型可以用于多种不同的任务。'''

There is much more that you can do with text generation models!\
For example, in-context learning (sometimes also called demonstration learning) is a technique where the model is provided with examples of the task it needs to perform within the prompt itself.\
This helps the model understand the task better and generate more accurate responses.\
For sentiment analysis, you can provide a few examples of sentences along with their sentiment labels in the prompt.\
The model will then use these examples to infer the sentiment of new sentences.

Also, constraining the output tokens can help guiding the model to generate expected outputs and make it easier to parse the output.